In [1]:
import json
import logging
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from wordcloud import WordCloud
from train_models import create_avg_dataset
from collections import OrderedDict
from os import listdir
from os.path import isfile, join
from sklearn.externals import joblib
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
%matplotlib inline

/home/ivan/Documentos/image-labeling-author-profiling/venv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [2]:
BASE_PATH_DATA = '/media/ivan/DDE/datasets_proyecto_mt'
train_data_csv_paths = ['/datasets/en_train.csv', './datasets/es_train.csv', './datasets/ar_train.csv']
train_labels_json_paths = ['./authors_labels/en_train_labels.json', './authors_labels/es_train_labels.json','./authors_labels/ar_train_labels.json']
X_dataframes = [None for i in range(len(train_data_csv_paths))]
y_dataframes = [None for i in range(len(train_labels_json_paths))]
list_of_ids = [None for i in range(len(X_dataframes))]
authors_ids = []
i = 0
t_start = time()
for path_x, path_y in zip(train_data_csv_paths, train_labels_json_paths):
    list_of_ids[i] = path_x.strip().split('/')[2][:2]
    X_dataframes[i] = pd.read_csv(BASE_PATH_DATA + path_x, sep='\s*,\s*', header=0, encoding='ascii', engine='python')
#     X, y_dataframes[i] = create_avg_dataset(X, path_y)
#     y_pred_dict = OrderedDict([("class", y_dataframes[i])])
#     X_dataframes[i] = pd.concat([X.reset_index(drop=True), pd.DataFrame.from_dict(y_pred_dict)], axis=1)
    i += 1
print("{} seconds to read csv's and create all dataframes".format(time() - t_start))

71.17650938034058 seconds to read csv's and create all dataframes


In [ ]:
labels = ["en", "es", "ar"]
# interesting_labels = set(["miniskirt", "velvet", "hair_spray", "maillot.1", "wig", "bikini", "stole", "brassiere", "scoreboard", "feather_boa", "bath_towel", "airliner", "silky_terrier", "bonnet", "bulletproof_vest", "maillot", "Model_T", "sarong", "overskirt", "shower_curtain", "hoopskirt", "lipstick", "plate_rack", "hand_blower", "Lhasa", "wig", "velvet", "bath_towel", "lipstick", "feather_boa", "bonnet", "face_powder", "stole", "tub", "milk_can", "bathtub", "overskirt", "golden_retriever", "Angora", "birdhouse", "fur_coat", "bikini", "Maltese_dog", "brassiere", "hair_spray", "lawn_mower", "ice_lolly", "maillot.1", "hair_slide", "chain_saw", "ballplayer", "velvet", "cradle", "brassiere", "broccoli", "lynx", "red-backed_sandpiper", "chiffonier", "Border_terrier", "lipstick", "miniskirt", "African_chameleon", "bath_towel", "safety_pin", "sea_urchin", "vase", "lighter", "football_helmet", "snorkel", "coyote", "ant", "pot", "shield", "military_uniform", "paddle"])
# interesting_labels = set(["brassiere", "velvet", "bath_towel", "lipstick"])
interesting_labels = set(["Windsor_tie"])
images_paths = []
# with open("authors_candidates_for_classification_brassiere.txt", "w") as file:
threshold = 0.7
for i in range(len(X_dataframes)):
#         file.write(labels[i] + "\n\n")
    for index, row in X_dataframes[i].iterrows():
        for label in interesting_labels:
            if row[label] > threshold and row['class'] == 'male':
#                     file.write("author_id = {} image_idx = {} class = {}\n".format(row['author_id'], index, row['class']))
#                     file.write("{} = {} \n".format(label, row[label]))
#                     file.write("{}\n\n".format(decode_predictions(np.array([row.drop(["author_id", "class"])]), top=3)[0]))
                images_paths.append(labels[i] + '/photo/' + row['author_id'])
# print(images_paths)

In [5]:
MODEL = VGG16(weights='imagenet', include_top=True)
# image = load_img('1cccf94761df573219c3ca96e8abbec1.8.jpeg', target_size=(224, 224))
# image = img_to_array(image)
# image = np.expand_dims(image, axis=0)
# image = preprocess_input(image)
# features = MODEL.predict(image)
# x = np.reshape(features, 7*7*512)
# print(len(x))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
import shutil, os
from os.path import isfile, join
BASE_DIR_IMAGES = '/media/ivan/5a98638f-057c-4499-87fa-d0d7b41f24b4/home/ivan/pan18-author-profiling-training-2018-02-27/'
# os.mkdir(BASE_PATH_DATA + '/images_bra')
for author_dir in set(images_paths):
    language = author_dir[:2]
    images = [f for f in os.listdir(BASE_DIR_IMAGES + author_dir)]
    for img in images:
        try:
            file = join(BASE_DIR_IMAGES, join(author_dir, img))
            image = load_img(file, target_size=(224, 224))
            image = img_to_array(image)
            image = np.expand_dims(image, axis=0)
            image = preprocess_input(image)
            features = MODEL.predict(image)
            decoded = [t[1] for t in decode_predictions(features, top=3)[0]]
            if 'windsor_tie' in decoded:
                shutil.copy(file, BASE_PATH_DATA + '/Windsor_tie/' + language)
        except Exception as e:
            print(e)
        


In [28]:
def evaluate_image_representation(image_representation):
    models_dir = './models/'
    models_files_paths = [join(models_dir, f) for f in listdir(models_dir) if isfile(join(models_dir, f))]
    loaded_models = [joblib.load(model_path) for model_path in models_files_paths]
    predicted_gender = []
    genders = {"female" : 0, "male" : 0}
    model_male_predictor = []
    model_female_predictor = []
    for model_idx in range(len(loaded_models)):
        models_split = models_files_paths[model_idx].replace(models_dir, "") 
        models_split = models_files_paths[model_idx].strip().split('_')
        ids_models = []
        for s in models_split:
            if s == "svm":
                break
            ids_models.append(s)
        ids_models[0] = ids_models[0][-2:]
#         print("Using trained model {}".format(ids_models))
        y = loaded_models[model_idx].predict(image_representation)[0]
        genders[y] += 1
        if y == "male":
            model_female_predictor.append("-".join(ids_models))
    if genders["female"] == 7:
        print("All models predicted: female")
    else:
        print("Excepto: {}".format(", ".join(model_female_predictor)))

In [34]:
img_dir = '/home/ivan/Escritorio/test_images/'
img_files = [join(img_dir, f) for f in listdir(img_dir) if isfile(join(img_dir, f))]
for image_path in img_files:
    print(image_path)
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)
    features = MODEL.predict(image)
    decoded = [t[1] for t in decode_predictions(features, top=3)[0]]
    print('Attributes (top 3): \n* {}'.format("\n* ".join(decoded)))
    print("Gender: female")
    evaluate_image_representation(features)

/home/ivan/Escritorio/test_images/36f46347bd3cbd53d4535574a316c9b8.2.jpeg
Attributes (top 3): 
* lipstick
* bikini
* brassiere
Gender: female
All models predicted: female
/home/ivan/Escritorio/test_images/4d04cd62a711c5a8229659b5b505e634.0.jpeg
Attributes (top 3): 
* cloak
* overskirt
* velvet
Gender: female
All models predicted: female
/home/ivan/Escritorio/test_images/4d2276bf535fd6ea6da0ae85c97b9ff7.8.jpeg
Attributes (top 3): 
* velvet
* doormat
* birdhouse
Gender: female
All models predicted: female
/home/ivan/Escritorio/test_images/106086df9c362aa4dcf4c824ada0dc05.1.jpeg
Attributes (top 3): 
* pillow
* coral_reef
* velvet
Gender: female
All models predicted: female
/home/ivan/Escritorio/test_images/3ed1ce5866f31b50a902121cd2e27e0d.0.jpeg
Attributes (top 3): 
* lipstick
* brassiere
* wig
Gender: female
All models predicted: female
/home/ivan/Escritorio/test_images/e62e683a17bc461c68b9bac8d7771820.0.jpeg
Attributes (top 3): 
* velvet
* vestment
* quilt
Gender: female
All models pred